In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,056 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-05 19:45:57--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.17MB/s    in 0.8s    

2022-12-05 19:45:59 (1.17 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Module-16").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [8]:
from pyspark import SparkFiles
url = "https://amazon-movies-database-1.s3.amazonaws.com/amazon_reviews_us_Sports_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...|2015-08-31 00:00:00|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...| 

### Create DataFrames to match tables

In [10]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
review_df.show()
     

+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|        review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130|Chicago Blackhawk...|2015-08-31 00:00:00|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679|Copag Poker Size ...|2015-08-31 00:00:00|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336|Baoer 223 5.56x45...|2015-08-31 00:00:00|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227|All 

In [11]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35669025|             1|
|   48198949|            30|
|   43355824|             6|
|   33014261|             6|
|   23493243|             1|
|   30717305|             4|
|   15714077|             2|
|    7854719|             1|
|   12761428|             2|
|   14127895|             1|
+-----------+--------------+
only showing top 10 rows



In [12]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0092702VE|Abu Garcia Revo S...|
|B00HZ9WTVC|Lee 90355 Cutter ...|
|B00WS4FRQ8|BEEWARE Targets -...|
|B00AKJVYYC|NEW Callaway RAZR...|
|B00DIPSOM0|GoSports Ladder T...|
|B0017LEUCU|Body Sport Exerci...|
|B00B59OMS6|Swiftwick SUSTAIN...|
|B00KC5SYLU|EliteTek RG-14 Fo...|
|B000GKUAB8|Multisport Helmet...|
|B001P80Z10|Izzo Golf Mini Mo...|
|B0063HVE18|Weston 30-0008-W ...|
|B00UR4HHDM|  Mission Multi-Cool|
|B000P3WQ86|Plano Pro-Max Sin...|
|B00005UDHY|Speedo Hydrospex ...|
|B00NZCXMUA|Fastfender Sail F...|
|B000YPIENQ|5.11 Tactical #73...|
|B00I3P5PM2|Tfo Gts Inshore G...|
|B009SPBB10|Twins Special Ful...|
|B00WIWYBZI|Black Anime Foam ...|
|B004SRA7GI|Moultrie Game Spy...|
+----------+--------------------+
only showing top 20 rows



In [13]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = review_df.select(["review_id","customer_id", "product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).drop_duplicates()
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-08-31|
| RT3UD0R7EUKEQ|   43669858|B006MO9EK0|     978519300| 2015-08-31|
| RFG4KD5HNO0JX|   10525400|B000GBSI9I|     538384332| 2015-08-31|
| RCDLHRR30ZIO1|   20962528|B007Z5JHWU|     789460419| 2015-08-31|
| RX7CYM6M0RUPR|    8021206|B00DERH1OO|     741432353| 2015-08-31|
|R1T0WKT388CGFQ|   35117849|B002G1INUK|     468220145| 2015-08-31|
|R1Q0F59MV1ZKWU|   45678551|B00MX5ZTDS|     619112676| 2015-08-31|
|R1C8EF9IGI1L1Z|   10341488|B004D7PTQQ|      65550835| 2015-08-31|
|R2EUPN57R242BF|   12646082|B00WJESQCE|     282064208| 2015-08-31|
| RQOGCKFF52ZQT|    4542850|B00Q02NEIM|     271142603| 2015-08-31|
| RMC1K94TBOZHZ|   10138996|B00KNBIGJS|     186734871| 2015-08-31|
|R357CWTGMFEUCY|   42150529|B012XLONUQ|     929038315| 2015-08

In [14]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.csjpucftpan9.us-east-1.rds.amazonaws.com:5432/database-1"
config = {"user":"postgres", 
          "password": "December7", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)